In [53]:
import pandas as pd
import numpy as np

import glob
import json
import time
import os
import re


from matplotlib import pyplot as plt

In [13]:
d = {'mgcr_inst_gen_ctrct_number': 'str', 'mejoravit_mtge_prdt_mark_type': 'str'}

In [14]:
df = pd.read_csv('t_pdm_mortgage_20220125.csv', dtype=d)

C:\Users\XMX2212_1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,15,16,17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
df.dtypes

mtge_info_orig_source_name        object
mgcr_request_register_date        object
entity_name                       object
state_name                        object
branch_division_name              object
mortgage_destination_name         object
MONTO                            float64
mgcr_inst_gen_ctrct_number        object
mejoravit_mtge_prdt_mark_type     object
mgcr_own_comm_network_type        object
mgcr_balance_goal_amount         float64
mgcr_operation_goal_number       float64
mgcr_sum_total_amount            float64
mgcr_oper_sum_total_number       float64
mgcr_weighted_tax_amount         float64
contract_register_branch_name     object
mortgage_line_type                object
mgcr_limit_range_desc             object
mortgage_total_balance_amount    float64
ppty_township_name                object
dtype: object

In [39]:
print(df.shape)
df.head(4)

(313079, 20)


,mtge_info_orig_source_name,mgcr_request_register_date,entity_name,state_name,branch_division_name,mortgage_destination_name,MONTO,mgcr_inst_gen_ctrct_number,mejoravit_mtge_prdt_mark_type,mgcr_own_comm_network_type,mgcr_balance_goal_amount,mgcr_operation_goal_number,mgcr_sum_total_amount,mgcr_oper_sum_total_number,mgcr_weighted_tax_amount,contract_register_branch_name,mortgage_line_type,mgcr_limit_range_desc,mortgage_total_balance_amount,ppty_township_name
0,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entre 25 y 50 %,NaN,NaN
1,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mas de 50 y hasta 60%,NaN,NaN
2,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,2. Adquisicion de vivienda,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mas de 80 y hasta 90%,NaN,NaN
3,Aforo,01/01/2020,Afirme,Aguascalientes,Bajio,8. Creditos Hipotecarios (incluye destinos del...,NaN,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Entre 25 y 50 %,NaN,NaN


In [20]:
cols = df.columns
cols

Index(['mtge_info_orig_source_name', 'mgcr_request_register_date',
       'entity_name', 'state_name', 'branch_division_name',
       'mortgage_destination_name', 'MONTO', 'mgcr_inst_gen_ctrct_number',
       'mejoravit_mtge_prdt_mark_type', 'mgcr_own_comm_network_type',
       'mgcr_balance_goal_amount', 'mgcr_operation_goal_number',
       'mgcr_sum_total_amount', 'mgcr_oper_sum_total_number',
       'mgcr_weighted_tax_amount', 'contract_register_branch_name',
       'mortgage_line_type', 'mgcr_limit_range_desc',
       'mortgage_total_balance_amount', 'ppty_township_name'],
      dtype='object')

In [21]:
int_cols = list(filter(lambda s: 'number' in s, cols))


In [24]:
schema_input = os.path.join('schema', 'pdmmortgage.input.schema')
with open(schema_input, 'r') as f:
    schema_raw = json.load(f)
fields = schema_raw['fields']

In [32]:
fields_not_null = list(filter(lambda f: isinstance(f['type'], str), fields))

In [33]:
fields_not_null

[{'name': 'mtge_info_orig_source_name',
  'type': 'string',
  'logicalFormat': 'ALPHANUMERIC(10)',
  'deleted': False,
  'metadata': False,
  'default': ''},
 {'name': 'mgcr_request_register_date',
  'type': 'string',
  'logicalFormat': 'DATE',
  'deleted': False,
  'metadata': False,
  'default': '',
  'format': 'yyyy-MM-dd',
  'locale': 'mx'},
 {'name': 'cutoff_date',
  'type': 'string',
  'logicalFormat': 'DATE',
  'deleted': False,
  'metadata': True,
  'default': '',
  'format': 'yyyy-MM-dd',
  'locale': 'mx'}]

In [38]:
c = 'entity_name'
idx = df[c].isnull()
idx_num = [i for i, x in enumerate(idx) if x]
df.loc[idx, c]

213492    NaN
213493    NaN
213494    NaN
213495    NaN
213496    NaN
         ... 
220568    NaN
220569    NaN
220570    NaN
220571    NaN
220572    NaN
Name: entity_name, Length: 95, dtype: object

In [43]:
df[c] = df[c].fillna('BanCoppel')

In [44]:
df[idx]

,mtge_info_orig_source_name,mgcr_request_register_date,entity_name,state_name,branch_division_name,mortgage_destination_name,MONTO,mgcr_inst_gen_ctrct_number,mejoravit_mtge_prdt_mark_type,mgcr_own_comm_network_type,mgcr_balance_goal_amount,mgcr_operation_goal_number,mgcr_sum_total_amount,mgcr_oper_sum_total_number,mgcr_weighted_tax_amount,contract_register_branch_name,mortgage_line_type,mgcr_limit_range_desc,mortgage_total_balance_amount,ppty_township_name
213492,Saldo,01/11/2021,BanCoppel,Aguascalientes,Bajio,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.420686,NaN
213493,Saldo,01/11/2021,BanCoppel,Guanajuato,Bajio,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.277097,NaN
213494,Saldo,01/11/2021,BanCoppel,Hidalgo,Bajio,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.483437,NaN
213495,Saldo,01/11/2021,BanCoppel,Queretaro,Bajio,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.142475,NaN
213496,Saldo,01/11/2021,BanCoppel,San Luis Potosi,Bajio,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.997566,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220568,Mercado,01/11/2021,BanCoppel,Yucatan,Sureste,2. Adquisicion de vivienda,0.389051,1,0,NaN,NaN,NaN,NaN,NaN,0.039294,NaN,NaN,NaN,NaN,NaN
220569,Mercado,01/11/2021,BanCoppel,Yucatan,Sureste,2. Adquisicion de vivienda,0.471078,2,0,NaN,NaN,NaN,NaN,NaN,0.047579,NaN,NaN,NaN,NaN,NaN
220570,Mercado,01/11/2021,BanCoppel,Yucatan,Sureste,8. Creditos Hipotecarios (incluye destinos del...,0.860129,3,0,NaN,NaN,NaN,NaN,NaN,0.086873,NaN,NaN,NaN,NaN,NaN
220571,Mercado,01/11/2021,BanCoppel,Zacatecas,Bajio,2. Adquisicion de vivienda,0.254342,1,0,NaN,NaN,NaN,NaN,NaN,0.025689,NaN,NaN,NaN,NaN,NaN


In [47]:

for c in map(lambda f: f['name'], fields_not_null):
    if c in df.columns:
        idx = df[c].isnull()
        print(idx.sum())
    else:
        print(f'{c} is not in df!')

0
0
cutoff_date is not in df!


In [48]:
encoding = 'utf-8-sig'
df.to_csv(f't_pdm_mortgage_20220125_{encoding}.csv', index=False, encoding=encoding)